In [6]:
import spacy
import pandas as pd
import numpy as np
from collections import Counter
from glob import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import re
from collections import Counter

In [7]:
nlp = spacy.load('en')
wap_raw = open('./input/war_and_peace.txt').read()
wap = nlp(wap_raw)

In [8]:
#filters words in text
words = []
for sent in wap.sents:
    line = sent.text
    for word in re.findall(r'\w+', line):
        words.append(word)
# top_words = Counter([w for w in words]).most_common(1000)
# print(top_words)
# words = list(top_words.elements())

In [9]:
#saves the filtered words into a new text file. Skip this if you already have the clean text
clean_wap = " ".join(words)
f = open('clean_wap.txt', 'w')
f.write(clean_wap)
f.close()

In [10]:
input_wap = LineSentence('./input/clean_wap.txt')

In [11]:
model_ted = Word2Vec(input_wap, size=100, window=5,
                    min_count=20, sg=1, workers=4, iter=15)

In [12]:
#filter words again
words = []
for sent in wap.sents:
    line = sent.text
    for word in re.findall(r'\w+', line):
        if word in model_ted.wv.vocab and word not in words:
            words.append(word)

In [13]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from sklearn.manifold import TSNE

In [14]:
words_top_vec_ted = model_ted[words]
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

Plotting the data

In [17]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)